In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dadaptation
import random


from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import h5py
import os
import sys
import pickle
import json

from utilities3 import *
from sklearn.model_selection import train_test_split
from Adam import Adam
from timeit import default_timer
from collections import OrderedDict

# import local modules from FNO2D.py
import FNO2D


import wandb
import datetime

### Define paths for data files

In [26]:
gc_path = "./Data/gc_array_filtered.npy"
d_path = "./Data/d_sol_array_filtered.npy"
coordinates_path = "./Data/Gc_out_coord.txt"

damage_coordinates_path = "./Data/d_out_coord.txt"

d_reduced_path = "./Data/d_reduced.npy"
gc_reduced_path = "./Data/gc_reduced.npy"

In [27]:
gc_ds_path = "./downsampled_data/gc_downsampled_data_array.npy"
d_ds_path = "./downsampled_data/d_downsampled_data_array.npy"
ds_coordinates_path = "./downsampled_data/gc_downsampled_data_coord.npy"

d_ds_reduced_path = "./downsampled_data/svd/d_reduced.npy"
gc_ds_reduced_path = "./downsampled_data/svd/gc_reduced.npy"

### Set project name, seed configurations and device configurations

In [17]:
# get device
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

# PROJECT_NAME = 'FNO2D'
PROJECT_NAME = 'PCA-NET'

# Set the random seeds to improve reproducibility by removing stochasticity
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False # Force cuDNN to use a consistent convolution algorithm
    torch.backends.cudnn.deterministic = True # Force cuDNN to use deterministic algorithms if available
    torch.use_deterministic_algorithms(True) # Force torch to use deterministic algorithms if available

set_seeds(0)

# for deterministic pytorch algorithms, enable reproducibility.
os.environ['CUBLAS_WORKSPACE_CONFIG']= ":4096:8"

# First create raw data of gc and d and upload to wandb

In [4]:
# read data from given path
gc = np.load(gc_path)
d = np.load(d_path)

# read coordinates csv file
coordinates = pd.read_csv(coordinates_path, sep = ",", header = None).values
damage_coordinates = pd.read_csv(damage_coordinates_path, sep = ",", header = None).values

In [5]:
# unit test for shape of data
assert len(gc) == len(d)
assert len(gc[0]) == len(coordinates)

# print out the shape of data
print("gc shape: ", gc.shape)
print("d shape: ", d.shape)
print("coordinates shape: ", coordinates.shape)
print("damage_coordinates shape: ", damage_coordinates.shape)


gc shape:  (2010, 13172)
d shape:  (2010, 6736)
coordinates shape:  (13172, 2)
damage_coordinates shape:  (6736, 2)


In [6]:
# create numpy array with shape (len(gc), gc.shape[1], 2)
input_mesh = np.zeros((len(gc), gc.shape[1], 2))
damage_mesh = np.zeros((len(gc), d.shape[1], 2))

# for each sample in input_mesh, add the coordinates
for i in range(len(gc)):
    input_mesh[i, :, :] = coordinates
    damage_mesh[i, :, :] = damage_coordinates
    
# add gc to the last dimension of input_mesh to have shape of (len(gc), gc.shape[1], 3)
gc_data = np.concatenate((input_mesh, np.expand_dims(gc, axis=2)), axis=2)
damage_data = np.concatenate((damage_mesh, np.expand_dims(d, axis=2)), axis=2)

In [7]:
# convert gc and d to hdf5 file
gc_hdf5 = h5py.File("./Data/gc_data.hdf5", "w")
gc_hdf5.create_dataset("gc_data", data = gc_data)
gc_hdf5.close()

d_hdf5 = h5py.File("./Data/damage_data.hdf5", "w")
d_hdf5.create_dataset("damage_data", data = damage_data)
d_hdf5.close()

In [35]:
with wandb.init(project="FNO2D", entity="jyyresearch", job_type="upload") as run:
    raw_data_at = wandb.Artifact("fracture-damage-raw-data", type="raw_data")
    raw_data_at.add_file("Data/gc_data.hdf5", name="gc_data")
    raw_data_at.add_file("Data/damage_data.hdf5", name="damage_data")
    run.log_artifact(raw_data_at)
    run.finish()

# Create reduced dataset of gc and d and upload to wandb

In [22]:
gc_reduced = np.load("./Data/gc_reduced.npy")
d_reduced = np.load("./Data/d_reduced.npy")

gc_reduced_hdf5 = h5py.File("./Data/gc_reduced.hdf5", "w")
gc_reduced_hdf5.create_dataset("gc_reduced", data = gc_reduced)
gc_reduced_hdf5.close()

d_reduced_hdf5 = h5py.File("./Data/d_reduced.hdf5", "w")
d_reduced_hdf5.create_dataset("d_reduced", data = d_reduced)
d_reduced_hdf5.close()

In [36]:
print("gc_reduced shape: ", gc_reduced.shape)
print("d_reduced shape: ", d_reduced.shape)

gc_reduced shape:  (2010, 377)
d_reduced shape:  (2010, 630)


In [9]:
with wandb.init(project=PROJECT_NAME, entity="jyyresearch", job_type="upload_reduced") as run:
    reduced_data_at = wandb.Artifact("fracture-damage-reduced-data", type="reduced_data")
    reduced_data_at.add_file("Data/gc_reduced.hdf5", name="gc_reduced")
    reduced_data_at.add_file("Data/d_reduced.hdf5", name="d_reduced")
    run.log_artifact(reduced_data_at)
    run.finish()

# Create DOWNSAMPLED data AND reduced dataset of gc and d and upload to wandb

In [28]:
# read data from given path
gc_data = np.load(gc_ds_path)
d_data = np.load(d_ds_path)
coordinates = np.load(ds_coordinates_path)

# unit test for shape of data
assert len(gc) == len(d)
assert len(gc[0]) == len(coordinates)

# print out the shape of data
print("gc shape: ", gc.shape)
print("d shape: ", d.shape)
print("coordinates shape: ", coordinates.shape)

gc shape:  (2010, 4786)
d shape:  (2010, 4786)
coordinates shape:  (4786, 2)


In [29]:
# convert gc and d to hdf5 file
gc_hdf5 = h5py.File("./downsampled_data/gc_data.hdf5", "w")
gc_hdf5.create_dataset("gc_data", data = gc_data)
gc_hdf5.close()

d_hdf5 = h5py.File("./downsampled_data/damage_data.hdf5", "w")
d_hdf5.create_dataset("damage_data", data = d_data)
d_hdf5.close()

In [30]:
with wandb.init(project="PCA-NET", entity="jyyresearch", job_type="upload") as run:
    raw_data_at = wandb.Artifact("fracture-damage-downsampled-raw-data", type="raw_data")
    raw_data_at.add_file("downsampled_data/gc_data.hdf5", name="gc_data")
    raw_data_at.add_file("downsampled_data/damage_data.hdf5", name="damage_data")
    run.log_artifact(raw_data_at)
    run.finish()

# Create reduced dataset for DOWNSAMPLED gc and d and upload to wandb

In [31]:
gc_reduced = np.load(gc_ds_reduced_path)
d_reduced = np.load(d_ds_reduced_path)

gc_reduced_hdf5 = h5py.File("./downsampled_data/gc_reduced.hdf5", "w")
gc_reduced_hdf5.create_dataset("gc_reduced", data = gc_reduced)
gc_reduced_hdf5.close()

d_reduced_hdf5 = h5py.File("./downsampled_data/d_reduced.hdf5", "w")
d_reduced_hdf5.create_dataset("d_reduced", data = d_reduced)
d_reduced_hdf5.close()

In [32]:
print("gc_reduced shape: ", gc_reduced.shape)
print("d_reduced shape: ", d_reduced.shape)

gc_reduced shape:  (2010, 379)
d_reduced shape:  (2010, 523)


In [33]:
with wandb.init(project="PCA-NET", entity="jyyresearch", job_type="upload_reduced") as run:
    reduced_data_at = wandb.Artifact("fracture-damage-downsampled-reduced-data", type="reduced_data")
    reduced_data_at.add_file("downsampled_data/gc_reduced.hdf5", name="gc_reduced")
    reduced_data_at.add_file("downsampled_data/d_reduced.hdf5", name="d_reduced")
    run.log_artifact(reduced_data_at)
    run.finish()

### We also upload the mesh of gc and d respectively for logging purposes

In [8]:
with wandb.init(project=PROJECT_NAME, entity="jyyresearch", job_type="upload_coordinates") as run:
    coordinates = wandb.Artifact("fracture-damage-coordinates", type="coordinates")
    coordinates.add_file("Data/gc_out_coord.txt", name="gc_out_coord")
    coordinates.add_file("Data/d_out_coord.txt", name="d_out_coord")
    run.log_artifact(coordinates)
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jyyresearch (use `wandb login --relogin` to force relogin)


### We also upload the mesh of DOWNSAMPLED gc and d respectively for logging purposes

In [34]:
with wandb.init(project="PCA-NET", entity="jyyresearch", job_type="upload_coordinates") as run:
    coordinates = wandb.Artifact("fracture-damage-downsampled-coordinates", type="coordinates")
    coordinates.add_file(ds_coordinates_path, name="ver_coord")
    run.log_artifact(coordinates)
    run.finish()